In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Point

In [ ]:
validation_file_osm_data_map = {"../assets/validation_data/teesta_bangladesh.geojson":"../assets/outputs/test_outputs/final_output/Bangladesh_final_dataset.parquet",
"../assets/validation_data/egypt_main_canals.geojson":"../assets/outputs/test_outputs/final_output/Egypt_final_dataset.parquet",
"../assets/validation_data/india_indira_gandhi_canals_mercator.geojson":"../assets/outputs/test_outputs/final_output/India_northern-zone_final_dataset.parquet",
"../assets/validation_data/krishna_godavari_delta_canals.geojson":"../assets/outputs/test_outputs/final_output/India_southern-zone_final_dataset.parquet",
"../assets/validation_data/sindh_pakistan_canals.geojson":"../assets/outputs/test_outputs/final_output/Pakistan_final_dataset.parquet",

}

In [ ]:
#fucntion to compute recall
def compute_recall(
        validation_path: str,
        grain_path: str,
        buffer_meters: float = 30,
        metric_crs: int | str = 3857
    ) -> float:
    """
    Recall = (length of validation canals that intersect a buffered GRAIN network)
             / (total length of validation canals)

    The GRAIN layer is buffered by `buffer_meters`, creating a corridor around each
    candidate canal.  Any portion of the validation network that lies within this
    corridor is treated as a “hit”.

    Parameters
    ----------
    validation_path : str
        File path to the *ground-truth* canal network (GeoJSON, Shapefile, GPKG …).
    grain_path : str
        File path to the GRAIN canal layer covering the same region.
    buffer_meters : float, default 30
        Distance tolerance for matching (in metres).
    metric_crs : int | str, default 3857
        Projected CRS with metre units (use a local UTM if possible).

    Returns
    -------
    float
        Recall in the range [0, 1].  Returns np.nan if the validation file has zero length.
    """
    # 1 ── Load data & project to a metric CRS 
    val   = gpd.read_file(validation_path).to_crs(metric_crs)
    grain = gpd.read_file(grain_path, bbox=tuple(val.total_bounds)).to_crs(metric_crs)
    
    # 2 ── Quick exit if either layer is empty
    total_val_len = val.length.sum()
    if total_val_len == 0 or grain.empty:
        return np.nan

    # 3 ── Buffer GRAIN lines
    grain_buf = grain.buffer(buffer_meters).unary_union         
    grain_buf_gdf = gpd.GeoDataFrame(geometry=[grain_buf], crs=metric_crs)

    # 4 ── Intersect validation canals with the buffered polygon
    intersected = gpd.overlay(val, grain_buf_gdf, how="intersection")
    matched_len = intersected.length.sum()

    recall = matched_len / total_val_len
    total_length_valdiation = val.length.sum()
    total_length_matched_grain = matched_len

    #grain canals = predicted calss = 'canals'
    grain_canals = grain[grain['predicted_class'].isin(['Canal_man_made', 'Canal_natural'])]   
    grain_canals_length = grain_canals.length.sum()
    # return all
    return {
        "recall": recall,
        "total_length_validation": total_length_valdiation/1E3,
        "total_length_matched_grain": total_length_matched_grain/1E3,
        "length_missing_grain": total_length_valdiation/1E3 - total_length_matched_grain/1E3,
        "total_length_grain_canals": grain_canals_length/1E3
    }


In [ ]:


def _nearest_index(pt, sindex):
    """
    Return index of the nearest geometry in `sindex` to `pt`, working on
    both old (rtree/pygeos) and new (Shapely-2 STRtree) back-ends.
    """
    try:
        # Newer GeoPandas (STRtree) – returns ndarray of ints
        idx_arr = sindex.nearest(pt)          # pt is a geometry
        return idx_arr[0]                     # take first match
    except TypeError:
        # Older back-ends – generator interface
        return next(sindex.nearest(pt, num_results=1))

def compute_positional_error(
        validation_path: str,
        grain_path: str,
        sample_spacing: float = 20,
        metric_crs: int | str = 3857
    ) -> dict:
    """
    Sample points every `sample_spacing` m along the validation canals and
    measure nearest-segment distance to the GRAIN network.

    Returns a dict: mean_dist, median_dist, p95_dist, npe, n_samples
    """
    # ── load & project ───────────────────────────────────────────────────────────
    val   = gpd.read_file(validation_path).to_crs(metric_crs)
    grain = gpd.read_file(grain_path, bbox=tuple(val.total_bounds)).to_crs(metric_crs)
    if val.empty or grain.empty:
        return {k: np.nan for k in
                ("mean_dist","median_dist","p95_dist","npe","n_samples")}

    sindex = grain.sindex

    # ── sample points along validation lines ────────────────────────────────────
    sample_pts = []
    for line in val.geometry:
        if line.is_empty or not line.is_valid:
            continue
        for d in np.arange(0, line.length + sample_spacing, sample_spacing):
            sample_pts.append(Point(line.interpolate(d)))

    if not sample_pts:
        return {k: np.nan for k in
                ("mean_dist","median_dist","p95_dist","npe","n_samples")}

    # ── compute distances ───────────────────────────────────────────────────────
    distances = []
    for pt in sample_pts:
        cand_idx   = _nearest_index(pt, sindex)        # geometry-aware lookup
        nearest_seg = grain.iloc[cand_idx].geometry
        distances.append(pt.distance(nearest_seg))

    #create a pandas DataFrame from distances
    distances_list = []

    for row in distances:
        distances_list.append(row[0])
    
    #drop any NaN values
    distances_list = [d for d in distances_list if isinstance(d, float)]
    distances_df = pd.DataFrame(distances_list)


    #remove any values not a float
    distances_df = distances_df[distances_df[0].apply(lambda x: isinstance(x, float))]

    total_len = val.length.sum()

    return {
        "total_length_validation": total_len,
        "mean_error_dist":   distances_df.mean(),
        "median_error_dist": distances_df.median(),
        "p95_error_dist":    distances_df.quantile(0.95),
        "pe": distances_df.mean()/total_len if total_len else np.nan,
        "npe":         (distances_df.sum() / total_len) if total_len else np.nan,
        "n_samples":   len(distances_df)
    }



In [ ]:
validation_region = validation_file_osm_data_map.keys()
test1_validation = list(validation_region)[3]
print(test1_validation)
test1_gpd = gpd.read_file(test1_validation)
test1_matching_grain = gpd.read_file(validation_file_osm_data_map[test1_validation])

/water3/saraths/Research_Files/GRAIN/assets/validation_data/krishna_godavari_delta_canals.geojson


In [ ]:
recall = compute_recall(
    validation_path=test1_validation,
    grain_path=validation_file_osm_data_map[test1_validation],  
    buffer_meters=500,
    metric_crs=3857
)



/tmp/ipykernel_3889626/2045550241.py:44: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  grain_buf = grain.buffer(buffer_meters).unary_union          # single (multi)-polygon


In [ ]:
recall

{'recall': np.float64(0.5718651367529091),
 'total_length_validation': np.float64(9404.201112380948),
 'total_length_matched_grain': np.float64(5377.934755183591),
 'length_missing_grain': np.float64(4026.266357197357),
 'total_length_grain_canals': np.float64(9669.539231433679)}

In [ ]:

positional_error_metric = compute_positional_error(
    validation_path=test1_validation,
    grain_path=validation_file_osm_data_map[test1_validation],  
    sample_spacing=500,
    metric_crs=3857
)

In [56]:
positional_error_metric

{'total_length_validation': np.float64(9404201.112380948),
 'mean_error_dist': 0    133682.125515
 dtype: float64,
 'median_error_dist': 0    133403.253543
 dtype: float64,
 'p95_error_dist': 0    172014.840726
 Name: 0.95, dtype: float64,
 'pe': 0    0.014215
 dtype: float64,
 'npe': 0    297.309854
 dtype: float64,
 'n_samples': 20915}